# Lakebase Database Setup

This notebook creates the database schema and populates it with sample coffee shop data.

In [2]:
import psycopg2
import logging
from databricks.sdk import WorkspaceClient
import uuid

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Configuration

In [ ]:
# Configuration
INSTANCE_NAME = "rb-demo-lakebase"
PROFILE = "az-demo"
USER_NAME = "rohit.bhagwat@databricks.com"  


In [ ]:
# Get instance and generate credentials
w = WorkspaceClient(profile=PROFILE)
instance = w.database.get_database_instance(name=INSTANCE_NAME)
cred = w.database.generate_database_credential(
    request_id=str(uuid.uuid4()), 
    instance_names=[INSTANCE_NAME]
)

print(f"✅ Connected to instance: {instance.name}")
print(f"   DNS: {instance.read_write_dns}")

## Connect to Database

In [ ]:
# Connect to PostgreSQL
conn = psycopg2.connect(
    host=instance.read_write_dns,
    dbname="databricks_postgres",
    user=USER_NAME,
    password=cred.token,
    sslmode="require"
)

conn.autocommit = True
print("✅ Connected to PostgreSQL database")

## Create Schema and Tables

In [ ]:
with conn.cursor() as cursor:
    # Create schema
    cursor.execute("CREATE SCHEMA IF NOT EXISTS coffee_operations")
    print("✅ Created schema: coffee_operations")
    
    # Create coffee shops table
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS coffee_operations.coffee_shops (
        shop_id VARCHAR(10) PRIMARY KEY,
        shop_name VARCHAR(100) NOT NULL,
        city VARCHAR(50) NOT NULL,
        state_province VARCHAR(50),
        country VARCHAR(50) NOT NULL,
        time_zone VARCHAR(50) NOT NULL,
        latitude NUMERIC(10,7),
        longitude NUMERIC(11,7),
        seating_capacity INTEGER,
        is_premium_location BOOLEAN DEFAULT FALSE,
        is_active BOOLEAN DEFAULT TRUE,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """
    cursor.execute(create_table_sql)
    print("✅ Created coffee_shops table")

## Insert Sample Data

In [ ]:
with conn.cursor() as cursor:
    # Sample coffee shop data
    sample_shops = [
        ('CS001', 'Brew & Bean Downtown', 'San Francisco', 'CA', 'United States', 
         'America/Los_Angeles', 37.7749, -122.4194, 45, True),
        ('CS002', 'Mocha Haven Central', 'New York', 'NY', 'United States', 
         'America/New_York', 40.7128, -74.0060, 32, True),
        ('CS003', 'Espresso Corner', 'Seattle', 'WA', 'United States', 
         'America/Los_Angeles', 47.6062, -122.3321, 28, False),
        ('CS004', 'Café Paris', 'Paris', 'Île-de-France', 'France', 
         'Europe/Paris', 48.8566, 2.3522, 50, True),
        ('CS005', 'Tokyo Roast', 'Tokyo', 'Tokyo', 'Japan', 
         'Asia/Tokyo', 35.6762, 139.6503, 35, False)
    ]
    
    # Insert data
    for shop in sample_shops:
        cursor.execute("""
            INSERT INTO coffee_operations.coffee_shops 
            (shop_id, shop_name, city, state_province, country, time_zone, 
             latitude, longitude, seating_capacity, is_premium_location)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (shop_id) DO NOTHING
        """, shop)
    
    print(f"✅ Inserted {len(sample_shops)} coffee shops")

## Verify Data

In [ ]:
with conn.cursor() as cursor:
    # Count total shops
    cursor.execute("SELECT COUNT(*) FROM coffee_operations.coffee_shops")
    total_count = cursor.fetchone()[0]
    print(f"📊 Total coffee shops: {total_count}")
    
    # Show premium locations
    cursor.execute("""
        SELECT shop_name, city, country, seating_capacity 
        FROM coffee_operations.coffee_shops 
        WHERE is_premium_location = TRUE
        ORDER BY seating_capacity DESC
    """)
    
    premium_shops = cursor.fetchall()
    print(f"\n🌟 Premium locations ({len(premium_shops)}):")
    for shop in premium_shops:
        print(f"   {shop[0]} - {shop[1]}, {shop[2]} ({shop[3]} seats)")

In [ ]:
# Close connection
conn.close()
print("\n✅ Database setup complete!")

## Next Steps

Your database is now ready! Proceed to the next notebook to query and analyze the data.